__The purpose of this notebook is to convert RMS's given data files into the format expected to be input in to the platform__
* Data is expected to be in the long form rather than wide form as we want to contrain to limited number of fields when the data exists as stored on the platform.
* Data is converted to wide format only when needed and processed in memory.
* Essentially, data input validation __must__ check to ensure that the input data is in the format of the output CSV
* Raise Error If data is not in this shape  
  


* __Raise Error__ When Data is in the Wrong Shape, Or there are Duplicates
    * Data is in the Wide form
    * There are duplicates of (Store,Wk,Item_ID)
* __Raise Warning__ if data is in right shape, but is incomplete
    * Some stores do not have complete set of all weeks of data
    * Some stores do not have all products (on some weeks)
    * In general if the long form doesn not have the number of rows = (# of Stores * # of Weeks * # of Products)


In [1]:
import pandas as pd
import os

__Configuration__

In [2]:
instore_data_path = 'data/Instore/'
instore_data_filenames = os.listdir(instore_data_path)

master_data_output_path = 'data/data_all.csv'

__Transformation__

In [3]:
master_data = pd.DataFrame(columns=['Wk','Store','Tier','Groups'])

# Merge All Data Files
for instore_data_fn in instore_data_filenames:
    master_data = pd.merge(left=master_data,
                          right=pd.read_csv(instore_data_path+instore_data_fn),
                          on=('Wk','Store','Tier','Groups'),
                          how='outer'
                         )

# Remove Erroneous Dashes in Column Names
master_data.columns = master_data.columns.str.replace("-", "")

# Convert Data to Machine Readable Long Form
master_data_long = pd.wide_to_long(df=master_data,
                         stubnames=["Qty_","Price_"],
                         i=["Wk","Tier","Groups","Store"],
                         j="Item_ID")


# Reset Index
master_data_long = master_data_long.reset_index()

# Remove Duplicates, Take Highest Non-Nan Value
master_data_long = master_data_long.sort_values(['Qty_'], ascending=False)
master_data_long_no_dupes = master_data_long.drop_duplicates(subset=['Store','Wk','Item_ID'], keep='first')

# Re-Order DataFrame
master_data_long_no_dupes = master_data_long_no_dupes.sort_values(['Store','Item_ID','Wk','Qty_','Price_'], ascending=False)

# Output to CSV
master_data_long_no_dupes.to_csv(master_data_output_path, index=None)

In [ ]:
master_data